In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, BitsAndBytesConfig
logg = lambda x: print(f"------------------------ {x} ---------------------------")
from datetime import datetime
print(torch.cuda.is_available())

True


In [2]:
ME = "/dpc/kunf0097/l3-8b"
RUN_ID = datetime.now().strftime("%y%m%d%H%M%S")
logg(RUN_ID)
logg('eval_simpler.py')

------------------------ 240627150602 ---------------------------
------------------------ eval_simpler.py ---------------------------


In [3]:
name = "amew0/l3-8b-medical-v240622193618"
tokenizer = AutoTokenizer.from_pretrained(
    name, 
    cache_dir=f"{ME}/tokenizer", 
    padding_side="right", 
    pad_token_id=0,
    legacy=False
)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Initialize model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
# Initialize model
model = AutoModelForCausalLM.from_pretrained(
    name, 
    cache_dir=f"{ME}/model",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"": 0},
    low_cpu_mem_usage=True,
    return_dict=True
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# Tokenize prompt function
cutoff_len = 128  # Maximum length to control CUDA OOM

def tokenize(prompt, tokenizer, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding="max_length",
        return_tensors="pt",
    )
    result["input_ids"] = result["input_ids"].flatten()
    result["attention_mask"] = result["attention_mask"].flatten()

    if add_eos_token and result["input_ids"].shape[0] < cutoff_len:
        result["input_ids"][-1] = tokenizer.eos_token_id
        result["attention_mask"][-1] = 1

    result["labels"] = result["input_ids"].clone()
    return result

def generate_and_tokenize_prompt(data_point):
    tokenized_full_prompt = tokenize(data_point["prompt"], tokenizer=tokenizer).to('cpu')
    return tokenized_full_prompt


In [6]:
data_path = './data/1/medical.json'
data = load_dataset("json", data_files=data_path)
eval_dataset = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [20]:
from sklearn.metrics import f1_score

# Evaluation function
def evaluate_squad_predictions(pred):
    labels = pred["labels"].flatten()
    logits = pred["logits"].argmax(-1).flatten()

    decoded_labels = tokenizer.decode(labels)
    decoded_logits = tokenizer.decode(logits)
    print(decoded_labels)
    print(decoded_logits)

    em = sum([1 if p == l else 0 for p, l in zip(labels, logits)]) / len(labels)
    f1 = f1_score(labels, logits, average='macro')

    return {'exact_match': em, 'f1': f1}

In [23]:
results = []
from tqdm import tqdm
for i, example in tqdm(enumerate(eval_dataset)):
    with torch.no_grad():
        example["input_ids"] = torch.LongTensor(example["input_ids"]).unsqueeze(0)
        example["attention_mask"] = torch.LongTensor(example["attention_mask"]).unsqueeze(0)

        outputs = model(input_ids=example["input_ids"], 
                        attention_mask=example["attention_mask"])
    pred = {
        "labels": example["input_ids"],
        "logits": outputs["logits"],
    }
    
    # Evaluate predictions
    eval_results = evaluate_squad_predictions(pred)
    results.append(eval_results)
    if i == 4: break


1it [00:00,  3.35it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|> If you are a doctor, please answer the medical questions based on the patient's description.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: Hello doctor..My name kamal I m 21 years / male.......l sufrred from a depression I m afraid when I m go to bed and couldn t sleep then everyday I thinking how I can sleep today then this illness is effect on my education then my health then cause no thing make me happy still nw I forget this feeling. When I have any work nextday I couldn t sleep and I m thinking what happen nextday then if I couldn t sleep How I
галіsystem글상위 If you are a doctor, please answer the medical questions based on the patient's description.글상위글상위user� This is the question: I,, I wife islesh k am 26 years old ..IIatelyfering from a severe and m taking of I m alone out sleep I I t sleep for I I m about to m sleep and and I is is very my my study and I marks and my of confidence to me happy

2it [00:00,  3.16it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|> If you are a doctor, please answer the medical questions based on the patient's description.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: I woke up last night with my heart racing. I was breathing slowly to calm my heart, but it kepts pumping extremely quickly. My depth perception is off and earlier my vision was blurry. I was shaking through all of this. Dad says its an anxiety attack.<|eot_id|><|start_header_id|>assistant<|end_header_id|> Hello, This can be anxiety attack but will not wake you up from the sleep, I am keeping it lower down. This can be arrhythmia like PSV or atrial fibrillation,
галіsystem글상위 If you are a doctor, please answer the medical questions based on the patient's description.글상위글상위user� This is the question: I have up this night with a heart racing and I was sweating fast and try myself heart down but it was racing racing. fast. I heart of was off, I today eyes was blurry. I

3it [00:01,  2.74it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|> If you are a doctor, please answer the medical questions based on the patient's description.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: I had gall bladder surgery two weeks ago. I was fine for five days and then I had an attach like I was having a heart attach, went to the hospital and given nitro and it put my heart rate over 140 and my blood pressure went high. They gave me a bunch of drugs to get rid of the symptoms, but the next morning I felt like my nerves where jumping out of my body. I have not been able to stop shaking since. I shake really bad
галіsystem글상위 If you are a doctor, please answer the medical questions based on the patient's description.글상위글상위user� This is the question: I have abl surgery  weeks ago. I have told for about days and then I started a episode. a was having a heart attack. I to the ER and they morphro and sent went me heart rate down 200. I blood pressure was to. I 

4it [00:01,  2.65it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|> If you are a doctor, please answer the medical questions based on the patient's description.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: I had surgery to fuse L4,L5 and S1 9 moths ago and I am experiencing pain in my lower back right side radiating around and into my groin. I had 3 steroid injections in my hip because a recent mri showed a tear in my hip socket. My orthopedist said it was nothing more than burcitis and that the pain I was having was to do with my back. My neuro surgeon say the fusion is fine and healing well.
галіsystem글상위 If you are a doctor, please answer the medical questions based on the patient's description.글상위글상위user� This is the question: I have a on remove my4-L5 and S1.2 months ago ago. have have still pain in my left back and side.ating down to down my butto. I have a2 epid injections in my back and of MRI MRIri showed a tear in my right lab. I hipopedic said that was not

4it [00:01,  2.19it/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|> If you are a doctor, please answer the medical questions based on the patient's description.<|eot_id|><|start_header_id|>user<|end_header_id|> This is the question: I fell down three stairs on my buttocks, when I got up I almost passed out I ran to my bed and waited till I felt better. I have been feeling nauseated since then and cant stand for a long period of time without feeling like im going to be sick. It is also painful to sit. Im on blood thinners what should I do?<|eot_id|><|start_header_id|>assistant<|end_header_id|> Dear France pain is from muscular strain due to injury. You can take analgesic like ibupro
галіsystem글상위 If you are a doctor, please answer the medical questions based on the patient's description.글상위글상위user� This is the question: I have on the steps and my leftcks and and I got up I could passed out. was to the bathroom and laid for I could better. I have a in betterated and then and I eat up very long t

In [9]:
# Aggregate metrics
em_total = sum(result['exact_match'] for result in results) / len(results)
f1_total = sum(result['f1'] for result in results) / len(results)

print(f"Aggregate EM: {em_total}, Aggregate F1: {f1_total}")

Aggregate EM: 0.0001796875, Aggregate F1: 4.9104171061135095e-05


In [16]:
with open(f'eval-{name[-12:]}.txt', 'w') as file:
    for result in results:
        file.write(f"{result}\n")
    file.write(f"{{'exact_match': {em_total}, 'f1': {f1_total}}}")

In [15]:
f"{{'exact_match': {em_total}, 'f1': {f1_total}}}"

"{'exact_match': 0.0001796875, 'f1': 4.9104171061135095e-05}"